# A Guide to Bitwise Operations & Shifts
We, as little children, were taught to count using the decimal number system. There isn't any definitive reason why we chose that but there are [some speculations](https://math.stackexchange.com/questions/317941/why-do-we-use-decimal-system). Regardless of that, some time in the past century, Computer Scientists, much like our budding selves, decided to use the binary number system for building computers but that wasn't as arbitrary a decision as our adoption of the decimal system.

This decision involved a lot of reasons - one of them that the 1s and 0s in the form of high or low voltages are easily distinguishable as opposed to measuring different voltages as different numbers to accomodate more digits (thus having a higher base than 2). Another was that of classifiying different voltages would require more circuitry and time. And while these are very high level (and not complete by any measure) reasons for why the base-2 (binary) system was chosen, we need not overwhelm or concern ourselves with those decisions yet, this chapter focuses more on the types of operations we can do with binary numbers.

When we talk about _operations_, we don't mean the rules laid out by BODMAS (or PEDMAS) including the operations defined in them like addition, multiplication, etc. We want to talk about the special **bitwise** operations that we can perform for optimizing our calculations.

Here's a little example, say we have to write the following Mathematical function in Python - <br><br>

$$ f(a,b) = a \cdot 2^b $$

In [1]:
def naive_f(a: int, b: int) -> int:
    return a * (2 ** b)

As the name implies, this is a **naive** implementation of the $f(a,b)$ function. But don't take my word for it. Let's collect some time data, so let us define a timeit function of our own.

In [2]:
import time
import typing


def timeit(f: typing.Callable, iterations: int = 200, args: typing.Tuple[typing.Any] = None) -> float:
    # Initialize start time.
    t1 = time.time()

    if args is not None:
        for _ in range(iterations):
            f(*args)
    else:
        for _ in range(iterations):
            f()
    
    # End time.
    t2 = time.time()

    return t2 - t1

In [3]:
# Tests for validity.
args_expected_vals: typing.Dict[typing.Tuple[int, int], int] = {
    (3, 8): 768,
    (7, 9): 3584,
    (2, 15): 65536,
    (2, 50): 2251799813685248,
    (30, 50): 33776997205278720,
    (7, 60): 8070450532247928832,
}

def valid(f: typing.Callable):
    for args, value in args_expected_vals.items():
        assert naive_f(*args) == value

valid(naive_f)

If we set $a = 7$ and $b = 60$, the function $f(a,b)$ should return $8070450532247928832$ which is quite a big number. So let's see how much time it takes to compute this 5 million times to see how efficiently we can do it.

In [4]:
t = timeit(naive_f, 5 * 10**6, (7, 60))
print(f'Took {t} seconds to compute naive_f(7, 60) 5 million times.')

Took 2.3578131198883057 seconds to compute naive_f(7, 60) 5 million times.


So, a little over 2 seconds. That's not too bad. But can we try to make it a little better? Let's try.

In [5]:
def fast_f(a: int, b: int) -> int:
    return a << b

If you didn't look closely enough, you might think that I'm comparing $a$ and $b$, or, more specifically, I'm checking if $a$ is less than $b$. But I'm actually performing a bitwise operation known as "left bitiwse". Or, I can say, I'm shifting $a$, $b$ bits to the left. That sounds weird but if we run our validity `valid` function on this, we can see it works just like our `naive_f` function!

In [6]:
valid(fast_f)

No assertion errors! But what about its efficiency? How does it actually compare with regards to speed to `naive_f`? Let's see.

In [7]:
t = timeit(fast_f, 5 * 10**6, (7, 60))
print(f'Took {t} seconds to compute fast_f(7, 60) 5 million times.')

Took 0.5532627105712891 seconds to compute fast_f(7, 60) 5 million times.


Just a little over half a second. That's pretty fast compared to our original implementation. But how? That's the question we aim to find in this chapter. We shall look at bitwise operations that can help us write faster code while also emphasizing real life use cases other than simple mathematical functions.

## What Are "Bitwise Operations"?
When I was learning about bitwise operations, I had no clue about what they were until a resource started with the comment "as the name suggests, bitwise operations are operations over individual bits". I don't remember if that was word to word but it really helped me to understand it. Even [Wikipedia](https://en.wikipedia.org/wiki/Bitwise_operation) describes it this way - 

> In digital computer programming, a bitwise operation operates on one or more bit patterns or binary numerals at the level of their individual bits.

There's a few different types of bitwise operations and we're here to talk about all of them. So let's list them all down and talk about each of them.

## Index
0. [Helper Functions](#Helper-Functions)
    1. [Binary Digits](#Binary-Digits)
    2. [Decimal Digits](#Decimal-Digits)
1. [Bitwise Operators](#Bitwise-Operators)
    1. [NOT Operator (~)](#NOT)
    2. [AND Operator (&)](#AND)
        1. [Brian Kernighan's Algorithm](#Brian-Kernighan's-Algorithm)
    3. [OR  Operator (|)](#OR)
    4. [XOR Operator (^)](#XOR)
2. [Bitwise Shifts](#Bitwise-Shifts)
    1. [Shifts](#Shifts)
        1. [Left Shift (<<)](#Left-Shift)
        2. [Right Shift (>>)](#Right-Shift)
    2. [Logical Shifts](#Logical-Shifts)
    3. [Arithmetic vs Logical Shifts](#Arithmetic-vs-Logical-Shifts)
3. [Miscellaneous](#Miscellaneous)
    1. [Closing Thoughts](#Closing-Thoughts)
    2. [Truth Table for Binary Operators](#Truth-Table)

## Helper Functions
To help us converting between 2's compliment binary representation of decimal digits back and forth, we need to have some helper function. We need not go into their implementation yet.
### Binary Digits
This method converts a given integer into a binary, 2's compliment string representation.

In [8]:
def bin_digits(n: int, bits: int = 8) -> str:
    s = bin(n & int("1"*bits, 2))[2:]
    return ("{0:0>%s}" % bits).format(s)

### Decimal Digits
This method is the inverse of the `bin_digits` function and is used to convert a binary string into an integer.

In [9]:
def dec_digits(n: str, bits: int = 8) -> int:
    while len(n) < bits:
        n = '0' + n
    if n.startswith('0'):
        return int(n, 2)
    else:
        return -1 * (int(''.join('1' if x == '0' else '0' for x in n), 2) + 1)

## Bitwise Operators
As described above bitwise operators are those operators that mutate or perform some action on the individual bits of a number represented in binary form. Broadly speaking, we can divide the above listed operations into two types - unary and binary although, theoretically speaking, we can have any _**n**-ary_ (formally known as "[arity](https://en.wikipedia.org/wiki/Arity)"). A C++ chapter on this topic is in development.

Here we'll talk about these binary operations and their mathematical formulae.

## NOT
$$ \newcommand{\Mod}[1]{\ \mathrm{mod}\ #1} \text{NOT }x = \sum_{n=0}^{\lfloor \log_{2}x \rfloor} 2^{n} \left[ \left( \left\lfloor \frac{x}{2^n} \right\rfloor \Mod{2} + 1 \right) \Mod{2} \right] = 2^{\lfloor \log_{2}x \rfloor + 1} - 1 -x $$
The operator is the only unary binary operator. It's sometimes also called the "negation operator". There's a lot of different notations for it but we, as Computer Scientists, will be using the _tilde_ notation, expressed as $ \text{~}x $. The NOT opeator, when applied over a proposition, simply negates it. It can be easily expressed in the form of a truth table - 

| $x$ | $\text{~}x$ |
|-----|-------------|
| $1$ |     $0$     |
| $0$ |     $1$     |

But what about NOT in terms of an bitwise operator? A bitiwise NOT performs the _inversion_ of bits over each bit in a number's two's compliment expansion. For example, take the number $54$. It can be represented as 00110110 in two's compliment binary notation. Performing NOT over each one of those digits would result in 11001001, which when converted back into decimal notation, is $-55$. Let's see it in action.

In [10]:
x = 54
bin_x = bin_digits(x)
bin_NOT_x = bin_digits(~x)
print(bin_x)
print(bin_NOT_x)
print(f'Converting {bin_x} to decimal:', dec_digits(bin_x))
print(f'Converting {bin_NOT_x} to decimal:', dec_digits(bin_NOT_x))

00110110
11001001
Converting 00110110 to decimal: 54
Converting 11001001 to decimal: -55


The NOT operator is the easiest to understand but the downside is that it's not a highly useful operator for any high level, practical reason. But still, pretty cool. The base-10 equivalent expression for the NOT operator is simply - <br><br>

$$ \text{NOT }x = -x - 1 = -(x + 1) $$

## AND
$$ x \text{ AND } y = \sum_{n=0}^{\lfloor \log_{2}x \rfloor} 2^{n} \left( \left\lfloor \frac{x}{2^n} \right\rfloor \Mod{2} \right) \left( \left\lfloor \frac{y}{2^n} \right\rfloor \Mod{2} \right) $$

The bitwise AND operator is a proper binary operator that requires two parameters. As the name implies, it performs the AND operation over each of the digits of the two numbers provided. The AND operation requires both the values it's being performed upon to be true (or 1 in our binary workings). It's truth table can be expressed as below - 

| $x$ | $y$ | $x \vee y$ |
|-----|-----|------------|
| $0$ | $0$ |     $0$    |
| $0$ | $1$ |     $0$    |
| $1$ | $0$ |     $0$    |
| $1$ | $1$ |     $1$    |

Once again, let us take an example. Take $x = 35$ and $y = 53$. An $\And$ operation over them would yield 33.

In [11]:
x = 35
y = 53
print(' ', bin_digits(x))
print('&', bin_digits(y))
print(' ', '-' * 8)
print('=', bin_digits(x & y))
print(f'\n{x} & {y} = {x & y}')

  00100011
& 00110101
  --------
= 00100001

35 & 53 = 33


In [12]:
for x_digit, y_digit in zip(bin_digits(x), bin_digits(y)):
    print(x_digit, '&', y_digit, '=', int(x_digit) & int(y_digit))

0 & 0 = 0
0 & 0 = 0
1 & 1 = 1
0 & 1 = 0
0 & 0 = 0
0 & 1 = 0
1 & 0 = 0
1 & 1 = 1


Bitwise AND is useful in something called "bit-masking". Bit masking is the process of applying a _mask_ to a number. A **mask** is specially constructed number to mutate the number in a specific way. For instance, say we want to make sure that the first three bits of a number are turned off (for reasons that are out of the scope of this chapter), we can construct a mask so that the first three digits are $0$s (we'll be working with 32 bit numbers).

In [13]:
mask = 0b11111000

Now we can apply this mask to any number we want using an $\And$ operator, regardless of what it has in its last three digits of its binary representation, the mask will convert them to $0$s. For example - 

In [14]:
for num in range(100):
    if not bin_digits(num & mask).endswith('000'):
        raise Exception('You broke logic!')
else:
    print('Valid.')

Valid.


Another use for the AND operator is to check for set bits. In the Factorial chapter, we have a function ($\sigma$) that checks for the number of set bits in a given natural number. We used a different approach in the chapter but now, empowered with the knowledge of the AND operator, we can write our own version.

## Brian Kernighan's Algorithm
I kind of lied in the last cell, because we won't really be writing our **original** set bit counting algorithm but will instead be looking over an efficient and popular one by a well known computer scientist named Brian Kernighan. This algorithms helps us count the number of 1s in the binary of a given number while iterating only the number of times it occurs. So for any number $n$, the runtime of this algorithm will be $O(\log{n})$.

|   Worst Case   |   Space  |
|----------------|----------|
| $O(\log{n})$   |  $O(1)$  |

The algorithms exploits the fact that any decimal number has all its bits flipped after its most significant bit. For example - 

In [15]:
for num in reversed(range(5, 10)):
    print(num, bin_digits(num))

9 00001001
8 00001000
7 00000111
6 00000110
5 00000101


So if we take a number and perform a bitwise AND with its decrement, in Maths terms - 

$$ n = n \text{ } \And \text{ } (n-1) $$

we get $n$ as the nunber with the rightmost set bit removed. And if we keep a counter that is incremented in a loop with each, we perform the above operation, we get the number of $1$s removed, and thus the total number of set bits of a number.

In [16]:
def count_set_bits(n: int) -> int:
    total = 0
    while n > 0:
        print(bin_digits(n), '&', bin_digits(n-1))
        n &= n - 1
        total += 1
    return total

In [17]:
count_set_bits(31)

00011111 & 00011110
00011110 & 00011101
00011100 & 00011011
00011000 & 00010111
00010000 & 00001111


5

## OR
$$ x \text{ OR } y = \sum_{n=0}^{\lfloor \log_2{x} \rfloor} 2^{n} \left( \left[ \left( \left\lfloor \frac{x}{2^n} \right\rfloor \Mod 2 \right) + \left( \left\lfloor \frac{y}{2^n} \right\rfloor \Mod 2 \right) + \left( \left\lfloor \frac{x}{2^n} \right\rfloor \Mod 2 \right)\left( \left\lfloor \frac{y}{2^n} \right\rfloor \Mod 2 \right) \right] \Mod 2 \right) $$

The OR operator is, similar to the AND operator, a binary operator. It requires two values, $x$ and $y$. The OR logical operator checks whether **at least one** of the values provided is True. If it is, then the entire expression evaluates to True. Thus the truth table for the OR logical operator can be defined as follows -

| $x$ | $y$ | $x \wedge y$ |
|-----|-----|------------|
| $0$ | $0$ |     $0$    |
| $0$ | $1$ |     $1$    |
| $1$ | $0$ |     $1$    |
| $1$ | $1$ |     $1$    |